In [1]:
import numpy as np
import copy
import tensorflow as tf
import keras
import keras.layers as L

Using TensorFlow backend.


In [2]:
class TicTacToe:
    def __init__(self):
        self.board = np.zeros(9)
        self.legal_actions = [action for action in range(0, 9)]
        self.finished = False
        self.player = 1
        
    def __str__(self):
        out = np.array(self.board).reshape((3, 3))
        return(str(out))
    
    def step(self, action):
        old_self = self.copy()
        
        self.board[action] = self.player
        self.legal_actions = [move for move in self.legal_actions if move != action]
        
        reward = self.reward()
        done = self.done()
        
        return old_self, action, reward, done
        
    def get_legal_actions(self):
        return(self.legal_actions)
        
    def get_legal_boards(self):
        boards = []
        for action in self.legal_actions:
            board = np.copy(self.board)
            board[action] = 1
            boards.append(board)
            
        return(boards, self.legal_actions)
    
    
    def flip_board(self):    
        self.board = -self.board
    
    def reward(self):
        player = self.player
        board = (self.board.reshape((3, 3)) == player) + 0
        
        if 3 in np.sum(board, axis = 0):
            return 1
        elif 3 in np.sum(board, axis = 1):
            return 1
        elif 3 is np.trace(board):
            return 1
        elif 3 is np.trace(board.T):
            return 1
        elif len(self.legal_actions) is 0:
            return 0
        return 0
    
    def done(self):
        player = self.player
        board = (self.board.reshape((3, 3)) == player) + 0
        if 3 in np.sum(board, axis = 0):
            return True
        elif 3 in np.sum(board, axis = 1):
            return True
        elif np.trace(board) == 3:
            return True
        elif np.trace(board.T) == 3:
            return True
        elif len(self.legal_actions) == 0:
            return True
        return False
    
    def reset(self):
        self.board = np.zeros(9)
        self.legal_actions = np.arange(0, 9)
        self.finished = False
        self.player = 1
        
    def __hash__(self):
        base3 = np.matmul(np.power(3, range(0, 9)), self.board.T)
        return int(base3)
    
    def copy(self):
        return copy.copy(self)
        

In [3]:
def generate_session():
    boards, actions, rewards = [], [], []
    
    board = TicTacToe()

    s = tf.InteractiveSession()
    s.run(tf.global_variables_initializer())



    while True:

        legal_boards, legal_actions = board.get_legal_boards()
        probs = get_action_prob(legal_boards)
        n_actions = probs.shape[0]
        probs = probs.reshape(n_actions)
        probs = probs / np.sum(probs)

        action = np.random.choice(np.arange(0, n_actions), 
                             p = probs)

        old_board, action, reward, done = board.step(legal_actions[action])

        #record session history to train later
        boards.append(board.board)
        actions.append(action)
        rewards.append(reward)
        board.flip_board()
        if done: break

    train_step(boards, actions, rewards)
            
    return sum(rewards)

In [4]:
def get_cumulative_rewards(rewards, #rewards at each step
                           gamma = 1 #discount for reward
                           ):
    """
    take a list of immediate rewards r(s,a) for the whole session 
    compute cumulative rewards R(s,a) (a.k.a. G(s,a) in Sutton '16)
    R_t = r_t + gamma*r_{t+1} + gamma^2*r_{t+2} + ...
    
    The simple way to compute cumulative rewards is to iterate from last to first time tick
    and compute R_t = r_t + gamma*R_{t+1} recurrently
    
    You must return an array/list of cumulative rewards with as many elements as in the initial rewards.
    """
    
    rewards = np.array(rewards)
    R = np.zeros_like(rewards, dtype= "float32")
    r = 0.
    for i, reward in enumerate(reversed(rewards)):
        r += reward
        R[-(i + 1)] = r
        r *= gamma
        
    return R
    
    

In [5]:
def train_step(_states, _actions, _rewards):
    """given full session, trains agent with policy gradient"""
    _cumulative_rewards = get_cumulative_rewards(_rewards)
    update.run({states_t: _states, actions_t: _actions, cumulative_rewards_t: _cumulative_rewards})

In [6]:
states_t  = tf.placeholder("float32", (None, 9), name = "states")
actions_t = tf.placeholder("int32", name = "action_ids")
cumulative_rewards_t = tf.placeholder("float32", name = "cumulative_rewards")

In [7]:
model = keras.models.Sequential()
model.add(L.Dense(32, activation = "relu"))
model.add(L.Dense(64, activation = "relu"))
model.add(L.Dense(1))

logits = model(states_t)
policy = tf.nn.softmax(logits)
log_policy = tf.nn.log_softmax(logits)

get_action_prob = lambda s: policy.eval({states_t: s})

J = tf.reduce_mean(log_policy * cumulative_rewards_t)

entropy = -tf.reduce_sum(tf.multiply(policy, log_policy), 1, name="entropy")

all_weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

loss = - J - 0.1 * entropy

update = tf.train.AdamOptimizer().minimize(loss,var_list=all_weights)
    
    

In [ ]:
s = tf.InteractiveSession()
s.run(tf.global_variables_initializer())

for i in range(100):
    
    rewards = [generate_session() for _ in range(10)] #generate new sessions
    
    print ("mean reward:%.3f"%(np.mean(rewards)))
        


In [11]:
board = TicTacToe()

legal_boards, legal_actions = board.get_legal_boards()
probs = get_action_prob(legal_boards)
n_actions = probs.shape[0]
probs = probs.reshape(n_actions)
probs = probs / np.sum(probs)

In [12]:
probs

array([0.11111111, 0.11111111, 0.11111111, 0.11111111, 0.11111111,
       0.11111111, 0.11111111, 0.11111111, 0.11111111], dtype=float32)

In [10]:
s.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES))

[array([[ 0.08963951, -0.0606088 ,  0.33861616, -0.19178468,  0.03264686,
          0.23321685,  0.21127781, -0.34392345, -0.09202439,  0.2920443 ,
         -0.28044686,  0.25361857, -0.20223527,  0.18872556,  0.30765715,
          0.35545877,  0.06304127, -0.12077245,  0.29229113, -0.18248522,
         -0.34825292,  0.24809936,  0.35049787,  0.20960578,  0.05717409,
          0.3232744 , -0.08902881,  0.17212132, -0.11147016,  0.01152691,
         -0.22509323, -0.36065575],
        [ 0.34798905, -0.21246557,  0.20837507,  0.22279641, -0.19324325,
         -0.15179949, -0.3581029 ,  0.34362367,  0.03008276,  0.23124465,
          0.15385911, -0.2628366 , -0.3174194 , -0.18199836, -0.08259496,
         -0.3472356 ,  0.22558185,  0.36895844,  0.03776971, -0.1015037 ,
          0.03571695, -0.2231832 ,  0.37499788,  0.23196152,  0.332307  ,
         -0.3698685 , -0.29020005,  0.36946717, -0.15301818, -0.02606788,
         -0.0940468 , -0.28661567],
        [ 0.08382541, -0.17899385, -0.18